## Crawling

In [1]:
import requests
import lxml.html
from selenium.webdriver import Chrome
import pandas as pd
import numpy as np

In [2]:
import re

In [3]:
def clean_text(text) : 
    return re.sub(r'[^ㄱ-ㅎ|가-힣|0-9|a-zA-Z]', '', text)

In [4]:
import time
from datetime import date, timedelta

In [5]:
def decrease_date(news_date) :
    t = time.strptime(news_date, '%Y%m%d')
    newdate = date(t.tm_year, t.tm_mon, t.tm_mday) -timedelta(1)
    return newdate.strftime('%Y%m%d')
    

In [2]:
main = "https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId={}&date={}"
sectionId = [101,102,103,104,105]
current_date = '20200501'

data = []
for i in range(100) :
    for j in sectionId :
        res = requests.get(main.format(j,current_date))
        root = lxml.html.fromstring(res.text)
        urls = root.cssselect('ol.ranking_list li div > a')
        for url in urls[:20] :
            res_1 = requests.get('http://news.naver.com' + url.attrib['href'])
            root_1 = lxml.html.fromstring(res_1.text)
            body = root_1.cssselect('div#articleBodyContents')
            try : 
                content = body[0].text_content()
                content = clean_text(content)
                data.append((current_date, content, j))
            except :
                continue
    current_date = decrease_date(current_date)
            

In [41]:
df = pd.DataFrame(data, columns=['date', 'contents','section'])

In [42]:
df.to_csv('news.csv', index = False, encoding = 'cp949')

In [6]:
df = pd.read_csv('news.csv', encoding = 'cp949')

In [7]:
df.head()

,date,contents,section
0,20200501,동영상 뉴스 오류를 우회하기 위한 ...,101
1,20200501,동영상 뉴스 오류를 우회하기 위한 ...,101
2,20200501,오류를 우회하기 위한 함수 추가 ...,101
3,20200501,오류를 우회하기 위한 함수 추가 ...,101
4,20200501,오류를 우회하기 위한 함수 추가 ...,101


In [8]:
def remove(x):
    x = re.sub(r'\s+',' ', x)
    first = x.find('추가')
    last = x.find('기자')
    return x[first+2: last-4]

In [9]:
df.contents = df.contents.apply(remove)

In [10]:
df.head()

,date,contents,section
0,20200501,앵커 오는 13일부터 전 국민에게 긴급 재난지원금을 나눠준다는데 이게 그냥 가만히...,101
1,20200501,앵커 오는 13일부터 전 국민에게 긴급 재난지원금을 나눠준다는데 이게 그냥 가만히...,101
2,20200501,미국 유명 래퍼 트래비스 스콧이 지난 24일 연 콘서트에 1230만 명이 몰렸다 ...,101
3,20200501,미국 유명 래퍼 트래비스 스콧이 지난 24일 연 콘서트에 1230만 명이 몰렸다 ...,101
4,20200501,근로장려세제처럼 환급형 세액공제로 해야 면세자는 기부해도 15 못 돌려받아저소득층...,101


In [11]:
df.drop_duplicates(subset=['contents'], inplace=True)

In [12]:
df.head()

,date,contents,section
0,20200501,앵커 오는 13일부터 전 국민에게 긴급 재난지원금을 나눠준다는데 이게 그냥 가만히...,101
2,20200501,미국 유명 래퍼 트래비스 스콧이 지난 24일 연 콘서트에 1230만 명이 몰렸다 ...,101
4,20200501,근로장려세제처럼 환급형 세액공제로 해야 면세자는 기부해도 15 못 돌려받아저소득층...,101
5,20200501,코로나19 항체검사 시행 시 숨겨진 감염자 확인 가능 국내도 코로나19 감염 모른...,101
7,20200501,머니투데이,101


In [13]:
df['contents'] = df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣|a-zA-Z]"," ")

In [14]:
df.head()

,date,contents,section
0,20200501,앵커 오는 일부터 전 국민에게 긴급 재난지원금을 나눠준다는데 이게 그냥 가만히...,101
2,20200501,미국 유명 래퍼 트래비스 스콧이 지난 일 연 콘서트에 만 명이 몰렸다 ...,101
4,20200501,근로장려세제처럼 환급형 세액공제로 해야 면세자는 기부해도 못 돌려받아저소득층...,101
5,20200501,코로나 항체검사 시행 시 숨겨진 감염자 확인 가능 국내도 코로나 감염 모른...,101
7,20200501,머니투데이,101


In [15]:
df['contents'].replace('', np.nan, inplace=True)
print(df.isnull().sum())

date        0
contents    1
section     0
dtype: int64


In [16]:
df.loc[df.contents.isnull()]

,date,contents,section
828,20200423,NaN,102


In [17]:
df = df.dropna(how = 'any')

In [18]:
df.loc[df.contents.isnull()]

,date,contents,section


In [19]:
df.to_csv('news_data.csv', index = False, encoding = 'cp949')